In [1]:
%reload_ext autoreload
%autoreload 2
%pdb off

import functools

import torch
from typing import Dict, List, Optional, Tuple, Union, TypeVar, Callable

import pyro
import pyro.distributions as dist

from causal_pyro.indexed.ops import IndexSet, gather, indices_of, scatter
from causal_pyro.interventional.handlers import do
from causal_pyro.counterfactual.handlers import MultiWorldCounterfactual, Preemptions

Automatic pdb calling has been turned OFF


In [25]:
def voting_model():
    u_vote0 = pyro.sample("u_vote0", dist.Bernoulli(0.6))
    u_vote1 = pyro.sample("u_vote1", dist.Bernoulli(0.6))
    u_vote2 = pyro.sample("u_vote2", dist.Bernoulli(0.6))
    u_vote3 = pyro.sample("u_vote3", dist.Bernoulli(0.6))
    u_vote4 = pyro.sample("u_vote4", dist.Bernoulli(0.6))

    vote0 = pyro.deterministic("vote0", u_vote0, event_dim=0)
    vote1 = pyro.deterministic("vote1", u_vote1, event_dim=0)
    vote2 = pyro.deterministic("vote2", u_vote2, event_dim=0)
    vote3 = pyro.deterministic("vote3", u_vote3, event_dim=0)
    vote4 = pyro.deterministic("vote4", u_vote4, event_dim=0)
    return {"outcome": vote0 + vote1 + vote2 + vote3 + vote4 >= 3}

# you might want the model to return other things, 
# or in the actual impact on multiple variables
# and you might prefer to pass a variable name and value as the consequent
# so dict here seems handy

In [62]:
#Q: what if we want multiple antecedents?

def preempt_with_factual(value: torch.Tensor, *, antecedent: str = "", event_dim: int = 0):
    factual_value = gather(value, IndexSet(**{antecedent: {0}}), event_dim=event_dim)
    return scatter({
        IndexSet(**{antecedent: {0}}): factual_value,
        IndexSet(**{antecedent: {1}}): factual_value,
    }, event_dim=event_dim)


In [55]:
antecedents = ["vote0"]
witness_candidates = [f"vote{i}" for i in range(1, 5)]

print(antecedents)
print(witness_candidates)

['vote0']
['vote1', 'vote2', 'vote3', 'vote4']


In [61]:

observations = dict(u_vote0=1., u_vote1=0., u_vote2=0., u_vote3=1., u_vote4=1.)

# @pyro.infer.infer_discrete(first_available_dim=-7)
# @pyro.infer.config_enumerate
@MultiWorldCounterfactual()
@do(actions=antecedents2)
@Preemptions(actions=preemptions)
@pyro.condition(data={k: torch.as_tensor(v) for k, v in observations.items()})
def ac_voting_model():
    consequent = voting_model()['outcome']
    intervened_consequent = gather(consequent, IndexSet(vote0={1}))
    observed_consequent = gather(consequent, IndexSet(vote0={0}))
    consequent_differs = intervened_consequent != observed_consequent
    pyro.factor("consequent_differs", torch.where(consequent_differs, torch.tensor(0.0), torch.tensor(-1e8)))
    print(indices_of(consequent), indices_of(consequent_differs))
    return intervened_consequent, observed_consequent, consequent_differs

print(ac_voting_model())

TypeError: preempt_with_factual() got an unexpected keyword argument 'antecedent'